Requêtage restaurant

In [71]:
import requests
from sqlalchemy import create_engine, text
import pandas as pd


with open('apikeyrm.txt', 'r') as file: #mettre clé api
    API_KEY = file.read().strip()


# LATITUDE = 43.604429
# LONGITUDE = 1.443812
#RADIUS = 5000  # en mètres
QUERY = 'chinois, japonais toulouse jean-jaures'


# URL de l'API Places

url2 = f"https://maps.googleapis.com/maps/api/place/textsearch/json?query={QUERY}&key={API_KEY}&sort_by=rating"
# Envoyer la requête

response2 = requests.get(url2)
# data = response.json()
data2 = response2.json()




Traitement de la requête 

In [72]:
nb_result = 20

# Utilisation de compréhension de dictionnaire pour créer le dictionnaire
dict_adresses = {
    'restaurant_id': [result['name'] + str(result['geometry']['location']['lat']) for result in data2['results'][:nb_result]],
    'name' : [result['name'] for result in data2['results'][:nb_result]],
    'formatted_address': [result['formatted_address'] for result in data2['results'][:nb_result]],
    'lat': [result['geometry']['location']['lat'] for result in data2['results'][:nb_result]],
    'lng' : [result['geometry']['location']['lng'] for result in data2['results'][:nb_result]],
    'photo_reference' : [result['photos'][0]['photo_reference'] for result in data2['results'][:nb_result]],
    'rating': [result['rating'] for result in data2['results'][:nb_result]],
    'user_ratings_total' : [result['user_ratings_total'] for result in data2['results'][:nb_result]],
}

# Affichage du dictionnaire pour vérification
print(dict_adresses)

name= [dict_adresses]

{'restaurant_id': ['THONG MINH MARKET43.6078445', 'Hinodé43.606462', 'Les Thermes Restaurant Chinois43.6045515', 'Zen-Saï43.6017016', 'Japoyaki43.6062214', "Jiao zi guan (l'art de ravioli)43.6057999", "Ni'shimai43.603", '米亚火锅 MIYA FONDUE43.599447', 'Restaurant Hop Seng43.6079411', 'YŪJŌ RAMEN ラーメン屋 TOULOUSE43.6036779', 'Mizuki Ramen みずき43.6020984', 'Fufu Ramen Toulouse43.6028711', 'King Fat Toulouse downtown43.6075274', 'Yoko Sushi43.6056491', 'Sakuraya Sushi43.6082273', 'MIAM MIAN ET SUSHI KINJO43.6021442', 'Wok to Walk Capitole43.60482409999999', 'Haozaï Restaurant43.6076176', 'Sushi Yoshi43.6006986', 'Asia Fast Food43.6082629'], 'name': ['THONG MINH MARKET', 'Hinodé', 'Les Thermes Restaurant Chinois', 'Zen-Saï', 'Japoyaki', "Jiao zi guan (l'art de ravioli)", "Ni'shimai", '米亚火锅 MIYA FONDUE', 'Restaurant Hop Seng', 'YŪJŌ RAMEN ラーメン屋 TOULOUSE', 'Mizuki Ramen みずき', 'Fufu Ramen Toulouse', 'King Fat Toulouse downtown', 'Yoko Sushi', 'Sakuraya Sushi', 'MIAM MIAN ET SUSHI KINJO', 'Wok to Wa

transformation requête en dataframe

In [73]:
df = pd.DataFrame(dict_adresses)
name_resto = df['name'].to_list()
lat_resto = df['lat'].to_list()




connection à la base de données

In [165]:
# Informations de connexion
DATABASE_TYPE = 'mysql'
DBAPI = 'pymysql'
HOST = 'localhost'
PORT = 3306
USER = 'root'
PASSWORD = 'Septembre_2024'
DATABASE = 'search_and_go'

# Chaîne de connexion
engine = create_engine(f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")


comparaison de la requête avec la base de données existante des restaurants

In [80]:
#Convertir la colonne "name" en liste
id_list = df["restaurant_id"].tolist()


#Générer les paramètres dynamiquement (:name0, :name1, etc.)
param_dict = {f"restaurant_id{i}": id for i, id in enumerate(id_list)}
placeholders = ", ".join(f":restaurant_id{i}" for i in range(len(id_list)))

#Requête SQL avec IN
query = text(f"SELECT * FROM restaurants WHERE restaurant_id IN ({placeholders})")

#Exécution de la requête
with engine.connect() as connection:
    result = connection.execute(query, param_dict)
    restaurants = result.fetchall()

restaurants = pd.DataFrame(restaurants)
display(restaurants)
if len(restaurants) !=0 :
    restaurants = set(restaurants["restaurant_id"].unique())  # Convertir en ensemble d'IDs
df_push = df.loc[~df["restaurant_id"].isin(restaurants)]
# Filtrer correctement

df_push

""


,restaurant_id,name,formatted_address,lat,lng,photo_reference,rating,user_ratings_total
0,THONG MINH MARKET43.6078445,THONG MINH MARKET,"15 Rue Denfert Rochereau, 31000 Toulouse, France",43.607844,1.448625,AVzFdbllpwflCfcThk_igOe7HGap3mkSi12Y575Ht_md7q...,4.4,577
1,Hinodé43.606462,Hinodé,"17 Rue d'Austerlitz, 31000 Toulouse, France",43.606462,1.447363,AVzFdbm2dDo-CCj4OKZ0LbV8RPdQWu0LzD_D9VVTXcX3Ec...,4.3,1145
2,Les Thermes Restaurant Chinois43.6045515,Les Thermes Restaurant Chinois,"44 Bd Lazare Carnot, 31000 Toulouse, France",43.604551,1.450263,AVzFdbnxADmIB6ITL32hVyfUiPSHYraE6xa6Ne7fMTvDcA...,4.3,861
3,Zen-Saï43.6017016,Zen-Saï,"2 Rue Jean Suau, 31000 Toulouse, France",43.601702,1.440094,AVzFdblby7WMdj8o1ekjvnt-kdngIoqJYhnB9fFLwFkfTF...,4.5,1161
4,Japoyaki43.6062214,Japoyaki,"14 Rue d'Austerlitz, 31000 Toulouse, France",43.606221,1.447532,AVzFdbnh2mR8D-mgEHqiLyBVrUDO05NDJ0oG1LRWgSroq1...,4.2,728
5,Jiao zi guan (l'art de ravioli)43.6057999,Jiao zi guan (l'art de ravioli),"13 Rue du Rem Villeneuve, 31000 Toulouse, France",43.605800,1.446345,AVzFdbk3ziVv6TA_TCZ5lQ0NEnzBi1OG6IEhpf_MxAuFWI...,4.9,736
6,Ni'shimai43.603,Ni'shimai,"2 Rue Joseph Lakanal, 31000 Toulouse, France",43.603000,1.441418,AVzFdbnvtzPMeQf7OJKFefrQVcLxUU5i-BMaRJJXb95RX4...,4.6,869
7,米亚火锅 MIYA FONDUE43.599447,米亚火锅 MIYA FONDUE,"d'Inde, 7 Rue du Coq d'Inde, 31000 Toulouse, F...",43.599447,1.442996,AVzFdblZ07SpQYG0z2OWR3XJk9Vv32mhNdJV6L72RYx184...,4.8,640
8,Restaurant Hop Seng43.6079411,Restaurant Hop Seng,"17 Rue Denfert Rochereau, 31000 Toulouse, France",43.607941,1.448790,AVzFdbmC9cv1m-Z6ufnHTh-ZJ5peHgmudXU8-XQnFvgXBa...,4.3,466
9,YŪJŌ RAMEN ラーメン屋 TOULOUSE43.6036779,YŪJŌ RAMEN ラーメン屋 TOULOUSE,"35 Rue Léon Gambetta, 31000 Toulouse, France",43.603678,1.442298,AVzFdbmV34Z86CywxwclDyiPUK9V15ip4uGyxCy20jQkiO...,4.6,1479


Envoie des restaurants vers la base de données

In [82]:
df_push.to_sql(name='restaurants', con=engine, if_exists = 'append', index=False)

20

verification et identification client

In [92]:
#entrée utilisateur

email = input('e-mail')

secret_pass = input('Password')



In [103]:
#verification idenfication


identification = 0

while identification == 0:

    email = input('e-mail')
    secret_pass = input('Password')
    query = text("SELECT * FROM user WHERE mail = :email AND password = :secret_pass")
        #Exécution de la requête
    with engine.connect() as connection:
        result = connection.execute(query, {'email': email, 'secret_pass': secret_pass})
        identification = result.fetchall()
    identification = pd.DataFrame(identification)
    display(identification)
    if len(identification) == 0 :
        display('Mail ou mot de passe incorrect')
        identification = 0
    else :
        identification = len(identification)

display('affichage page principale')

    

""


'Mail ou mot de passe incorrect'

,id,nom,prenom,mail,password
0,1,Martin,Jean,jean.martin@test.com,@test.com


'affichage page principale'

In [139]:
import re

new_user_mail = input('e-mail')
new_user_name = input('Name')
new_user_lastname = input('Lastname')
new_user_secret_pass = input('Password')

def checking(new_user_mail, new_user_name, new_user_lastname, new_user_secret_pass):
    #regex de qui vont servir de verification
    regex_mail = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    regex_password = r'^(?=.*[a-zàâäéèêëîïôöùûüÿçáéíñóúü])(?=.*[A-ZÀÂÄÉÈÊËÎÏÔÖÙÛÜŸÇÁÉÍÑÓÚÜ])(?=.*\d)(?=.*[@$!%*?&_\"\'\'])[A-Za-zàâäéèêëîïôöùûüÿçáéíñóúüÀÂÄÉÈÊËÎÏÔÖÙÛÜŸÇÁÉÍÑÓÚÜ\d@$!%*?&_\"\'\']{8,}$'
    query = text("SELECT * FROM user WHERE mail = :new_user_mail")
    with engine.connect() as connection:
            result = connection.execute(query, {'new_user_mail': new_user_mail})
            creation = result.fetchall()
    creation = pd.DataFrame(creation)
    if len(creation) == 0:
        creation.to_sql(name='user', con=engine, if_exists = 'append', index=False)
    else :
        return "email déjà connu merci de vous identifier"  
    if re.match(regex_mail, new_user_mail):
        new_user_mail = True
        if len(new_user_name) !=0:
            new_user_name = True
            if len(new_user_lastname) !=0:
                new_user_lastname = True
                if re.match(regex_password, new_user_secret_pass):
                    new_user_secret_pass = True
                else:
                    new_user_secret_pass = False
            else:
                new_user_lastname = False
        else:
            new_user_name = False  
    else:
        new_user_mail = False
    chek_in = [new_user_mail, new_user_name, new_user_lastname, new_user_secret_pass]
    return chek_in

checking(new_user_mail, new_user_name, new_user_lastname, new_user_secret_pass)



[True, True, True, True]

In [128]:
import re

# Exemple de mot de passe à tester
password = '"rèsT"658!'

# Expression régulière pour valider un mot de passe
regex = r'^(?=.*[a-zàâäéèêëîïôöùûüÿçáéíñóúü])(?=.*[A-ZÀÂÄÉÈÊËÎÏÔÖÙÛÜŸÇÁÉÍÑÓÚÜ])(?=.*\d)(?=.*[@$!%*?&_\"\'\'])[A-Za-zàâäéèêëîïôöùûüÿçáéíñóúüÀÂÄÉÈÊËÎÏÔÖÙÛÜŸÇÁÉÍÑÓÚÜ\d@$!%*?&_\"\'\']{8,}$'

# Vérification du mot de passe
if re.match(regex, password):
    print("Mot de passe valide")
else:
    print("Mot de passe invalide")


Mot de passe valide


In [187]:
class Checker:
    email_regex = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    password_regex = r'^(?=.*[a-zàâäéèêëîïôöùûüÿçáéíñóúü])(?=.*[A-ZÀÂÄÉÈÊËÎÏÔÖÙÛÜŸÇÁÉÍÑÓÚÜ])(?=.*\d)(?=.*[@$!%*?&_\"\'\'])[A-Za-zàâäéèêëîïôöùûüÿçáéíñóúüÀÂÄÉÈÊËÎÏÔÖÙÛÜŸÇÁÉÍÑÓÚÜ\d@$!%*?&_\"\'\']{8,}$'

    def __init__(self, engine):
        self.engine = engine

    def is_email_known(self, email):
        query = text("SELECT COUNT(*) FROM user WHERE mail = :email")
        with self.engine.connect() as connection:
            result = connection.execute(query, {'email': email})
            count = result.scalar()
        return count > 0

    @staticmethod
    def is_valid_email(email):
        return re.match(Checker.email_regex, email) is not None

    @staticmethod
    def is_valid_password(password):
        return re.match(Checker.password_regex, password) is not None

    @staticmethod
    def is_valid_name(name):
        return bool(name) and all(char.isalpha() or char.isspace() for char in name)

    @staticmethod
    def is_valid_lastname(lastname):
        return bool(lastname) and all(char.isalpha() or char.isspace() for char in lastname)

    def validate_user(self, email, password, name, lastname):
        if not self.is_valid_email(email):
            print("Mail invalide")
            return False
        if not self.is_valid_password(password):
            print("Mot de passe invalide merci de renseigner minimum 8 caractères dont un chiffre, majuscule et symbole")
            return False
        if not self.is_valid_name(name):
            print("Name invalid")
            return 
        if not self.is_valid_lastname(lastname):
            print("Lastname invalid")
            return 
        return True
    # alternatif
    def val(self, email, password, name, lastname):
        return not self.is_valid_email(email) and not self.is_valid_password(password) and not self.is_valid_name(name) and not self.is_valid_lastname(lastname)

    @staticmethod
    def add_user(email, password, name, lastname):
        dict_user = {'nom': [name], 'prenom': [lastname], 'mail': [email], 'password': [password]}
        df_user = pd.DataFrame(dict_user)
        df_user.to_sql(name='user', con=engine, if_exists = 'append', index=False)

# Exemple d'utilisation
email = "r.meson@test.com"
password = "WCS_Rémi_2025"
name = "Jean"
lastname = "Dupont"



In [188]:
checker = Checker(engine)

if checker.is_email_known(email):
    print("Mail déjà connu merci de vous identifier")
else:
    if checker.validate_user(email, password, name, lastname):
        checker.add_user(email, password, name, lastname)